In [5]:
import os
import sys
import shutil
import torch
import torchaudio
import numpy as np
import torchvision.transforms.v2 as transforms

from torchaudio.transforms import *
from fastai.vision.all import Image

In [ ]:
! git clone https://github.com/xpaf/MusicNet

In [9]:
from importlib.machinery import SourceFileLoader
preprocessing = SourceFileLoader('a', "/content/MusicNet/music_preprocessing/preprocessing.py").load_module()

In [12]:
GTZAN_ORIGINAL_DIR="/content/drive/MyDrive/Colab Notebooks/GSN/genres_original"
GTZAN_SPECTROGRAMS_DIR="gtzan_spectrograms"
SPECTROGRAM_TRANSFORMS = [
    MelSpectrogram(sample_rate=22050, n_fft=1024, f_min=0, f_max=10000, n_mels=128),
    AmplitudeToDB()
]
AUDIO_SLICES = 6

In [ ]:
# Remove corrupted file
if (os.path.isfile(GTZAN_ORIGINAL_DIR + "/jazz/jazz.00054.wav")):
  os.remove(GTZAN_ORIGINAL_DIR + "/jazz/jazz.00054.wav")

if (os.path.isdir(GTZAN_SPECTROGRAMS_DIR)):
  shutil.rmtree(GTZAN_SPECTROGRAMS_DIR)

os.mkdir(GTZAN_SPECTROGRAMS_DIR)

mel_spec_transformer = transforms.Compose(SPECTROGRAM_TRANSFORMS)
apply_idx_to_file = AUDIO_SLICES > 1

for genre in os.listdir(GTZAN_ORIGINAL_DIR):
  genre_original_dir = os.path.join(GTZAN_ORIGINAL_DIR, genre)
  spectrogram_genre_dir = os.path.join(GTZAN_SPECTROGRAMS_DIR, genre)
  os.mkdir(spectrogram_genre_dir)

  print(genre_original_dir + " -> " + spectrogram_genre_dir)

  for file in os.listdir(genre_original_dir):
    wave = preprocessing.load_audio_and_cut_length(os.path.join(genre_original_dir, file))
    waves = preprocessing.slice_audio(wave, AUDIO_SLICES)

    for idx, w in enumerate(waves):
      spectrogram = preprocessing.create_spectrogram_image(mel_spec_transformer, w)
      preprocessing.save_spectrogram_as_image(spectrogram, preprocessing.create_file_name(file, spectrogram_genre_dir, apply_idx_to_file, idx))

shutil.make_archive(GTZAN_SPECTROGRAMS_DIR, 'zip', GTZAN_SPECTROGRAMS_DIR)

/content/drive/MyDrive/Colab Notebooks/GSN/genres_original/classical -> gtzan_spectrograms/classical
